# SPML HW4: Model Extraction

In this notebook you'll explore model extraction.

In [7]:
######### Make sure to put your info #########
name = 'Fateme Raeijian'
std_id = '402203389'
##############################################

In [28]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from tqdm import tqdm, trange
import matplotlib.pyplot as plt


import torchvision
from torchvision import transforms, datasets, models
from torchvision.models import ResNet18_Weights, ResNet34_Weights  # Import weights enums


# Define device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Loading CIFAR100 (5 points)

Load the `CIFAR100` dataset. Make sure you resize the images to be `224x224` (same as the input size of resnet).

In [ ]:
# TODO: Load CIFAR-100 dataset

# Define transforms for training and testing
transform_train = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5071, 0.4865, 0.4409],
                         std=[0.2673, 0.2564, 0.2762]),
])

transform_test = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5071, 0.4865, 0.4409],
                         std=[0.2673, 0.2564, 0.2762]),
])

# Load the training and test sets
train_dataset = datasets.CIFAR100(root='./data', train=True, download=True, transform=transform_train)
test_dataset = datasets.CIFAR100(root='./data', train=False, download=True, transform=transform_test)

# Define data loaders
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True, num_workers=4)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False, num_workers=4)


Files already downloaded and verified
Files already downloaded and verified


# Pre-trained ResNet34 (10 points)

Load a pre-trained ResNet34 and train it on the `CIFAR100` dataset.

In [29]:
# TODO: Load pretrained ResNet-34 model

# Load the pre-trained ResNet-34 model
# resnet34 = models.resnet34(pretrained=True)
resnet34 = models.resnet34(weights=ResNet34_Weights.IMAGENET1K_V1)

# Modify the final fully connected layer to match CIFAR100 classes
num_ftrs = resnet34.fc.in_features
resnet34.fc = nn.Linear(num_ftrs, 100)

# Move the model to the appropriate device
resnet34 = resnet34.to(device)


In [30]:
# TODO: Train the model on CIFAR100

# Define loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(resnet34.parameters(), lr=0.001)

# Number of epochs
epochs = 10

# Training loop
for epoch in range(epochs):
    resnet34.train()  # Set model to training mode
    running_loss = 0.0
    
    # Use tqdm to wrap the DataLoader for a progress bar
    for inputs, labels in tqdm(train_loader, desc=f'Epoch {epoch+1}/{epochs}'):
        inputs, labels = inputs.to(device), labels.to(device)  # Move data to device
        
        optimizer.zero_grad()  # Zero the parameter gradients
        
        outputs = resnet34(inputs)  # Forward pass
        loss = criterion(outputs, labels)  # Compute loss
        loss.backward()  # Backward pass
        optimizer.step()  # Optimize
        
        running_loss += loss.item()  # Accumulate loss
    
    avg_loss = running_loss / len(train_loader)  # Average loss for the epoch
    print(f'Epoch [{epoch+1}/{epochs}], Loss: {avg_loss:.4f}')
    
    # Validation phase
    resnet34.eval()  # Set model to evaluation mode
    correct = 0
    total = 0
    
    with torch.no_grad():  # Disable gradient computation
        for inputs, labels in test_loader:
            inputs, labels = inputs.to(device), labels.to(device)  # Move data to device
            outputs = resnet34(inputs)  # Forward pass
            _, predicted = torch.max(outputs.data, 1)  # Get predictions
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    
    accuracy = 100 * correct / total  # Compute accuracy
    print(f'Validation Accuracy: {accuracy:.2f}%')

# Save the trained model
torch.save(resnet34.state_dict(), 'resnet34_cifar100.pth')
print("Model saved as resnet34_cifar100.pth")



Epoch 1/10:   0%|          | 0/782 [00:00<?, ?it/s]

Epoch 1/10: 100%|██████████| 782/782 [06:48<00:00,  1.92it/s]

Epoch [1/10], Loss: 2.2702


Validation Accuracy: 47.31%


Epoch 2/10: 100%|██████████| 782/782 [06:43<00:00,  1.94it/s]

...

Epoch [9/10], Loss: 0.3318


Validation Accuracy: 71.71%


Epoch 10/10: 100%|██████████| 782/782 [06:56<00:00,  1.88it/s]

Epoch [10/10], Loss: 0.2815


Validation Accuracy: 70.10%
Model saved as resnet34_cifar100.pth


You might want to save this model to avoid retraining.

In [33]:
torch.save(resnet34.state_dict(), 'resnet34_cifar100.pth')
print("Model saved as resnet34_cifar100.pth")

Model saved as resnet34_cifar100.pth


# Model Extraction (20 points)

Here we use knowledge distillation to extract models. If you are confused after the instructions take a look at the next section to understand what we are trying to do. The general steps in Knowledge Distillation are as follows:

1. Set the victim (teacher) to evaluation mode and the attacker (student) to training mode.
2. Use the victim to find the logits for each batch of inputs.
3. Predict the attackers output for the same batch of inputs.
4. Define and reduce the loss function over the difference between logits from the victim and attacker (use KL-Divergence, ...)
5. Repeat steps 2-4 for the number of epochs.

Feel free to check out [Distilling the Knowledge in a Neural Network](https://arxiv.org/abs/1503.02531) to get a better sense of the process.

In [34]:
def knowledge_distillation(victim_model, attacker_model, loader, optimizer, epochs, T):
    """
    Perform knowledge distillation from victim_model to attacker_model.

    Parameters:
    - victim_model: The pre-trained teacher model.
    - attacker_model: The student model to be trained.
    - loader: DataLoader for training data.
    - optimizer: Optimizer for the student model.
    - epochs: Number of training epochs.
    - T: Temperature parameter.
    """
    victim_model.eval()      # Set teacher to evaluation mode
    attacker_model.train()  # Set student to training mode

    criterion = nn.KLDivLoss(reduction='batchmean')
    softmax = nn.Softmax(dim=1)
    log_softmax = nn.LogSoftmax(dim=1)

    for epoch in range(epochs):
        running_loss = 0.0

        # Use tqdm to wrap the DataLoader for a progress bar
        for inputs, _ in tqdm(loader, desc=f'Distillation Epoch {epoch+1}/{epochs}'):
            inputs = inputs.to(device)

            with torch.no_grad():
                teacher_logits = victim_model(inputs)
                teacher_probs = softmax(teacher_logits / T)

            student_logits = attacker_model(inputs)
            student_log_probs = log_softmax(student_logits / T)

            loss = criterion(student_log_probs, teacher_probs)

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            running_loss += loss.item()

        avg_loss = running_loss / len(loader)
        print(f'Distillation Epoch [{epoch+1}/{epochs}], Loss: {avg_loss:.4f}')


Can you explain how we should set the temperature? Why is this choice appropriate for model extraction?

`your response:`
In knowledge distillation, the temperature parameter T is used to soften the probability distributions produced by the teacher and student models. A higher temperature smooths the probability distribution, making the differences between classes more apparent and capturing the relative probabilities of incorrect classes. Typically, a temperature between 2 and 10 is chosen.

For model extraction, setting a higher temperature (e.g., T=4) is appropriate because it allows the student (attacker) model to learn not only the hard labels but also the soft targets provided by the teacher (victim) model. This facilitates the transfer of more nuanced information, enhancing the student's ability to mimic the teacher's behavior more accurately, which is crucial for successful model extraction.

# Attack Transferability (20 points)

Implement attacks such as FGSM or PGD, you can use code from previous homeworks or readily available libraries.

In [35]:
# TODO: Load or implement attacks

import torch.nn.functional as F

def fgsm_attack(model, loss_fn, images, labels, epsilon):
    images = images.clone().detach().to(device)
    labels = labels.clone().detach().to(device)
    
    images.requires_grad = True
    outputs = model(images)
    loss = loss_fn(outputs, labels)
    model.zero_grad()
    loss.backward()
    data_grad = images.grad.data
    
    perturbed_images = images + epsilon * data_grad.sign()
    perturbed_images = torch.clamp(perturbed_images, 0, 1)
    
    return perturbed_images

def pgd_attack(model, loss_fn, images, labels, epsilon, alpha, iters):
    ori_images = images.clone().detach().to(device)
    images = images.clone().detach().to(device)
    
    for i in range(iters):
        images.requires_grad = True
        outputs = model(images)
        loss = loss_fn(outputs, labels)
        model.zero_grad()
        loss.backward()
        data_grad = images.grad.data
        
        images = images + alpha * data_grad.sign()
        eta = torch.clamp(images - ori_images, min=-epsilon, max=epsilon)
        images = torch.clamp(ori_images + eta, 0, 1).detach()
    
    return images


Fill in the following function to attack a model and report the accuracy of the victim on the adversarial examples generated using the available model.

In [36]:
# TODO: Fill in the transferability_attack function

def transferability_attack(model, victim, loader, attack, epsilon=0.03, alpha=0.007, iters=10):
    """
    Perform adversarial attack using the attacker model and evaluate on the victim model.

    Parameters:
    - model: The attacker model used to generate adversarial examples.
    - victim: The victim model to be evaluated on adversarial examples.
    - loader: DataLoader for the dataset to attack.
    - attack: The attack method ('fgsm' or 'pgd').
    - epsilon: Perturbation magnitude.
    - alpha: Step size for PGD.
    - iters: Number of iterations for PGD.

    Returns:
    - Accuracy of the victim model on adversarial examples.
    """
    victim.eval()
    model.eval()
    correct = 0
    total = 0
    loss_fn = nn.CrossEntropyLoss()
    
    for inputs, labels in loader:
        inputs, labels = inputs.to(device), labels.to(device)
        
        if attack == 'fgsm':
            adv_images = fgsm_attack(model, loss_fn, inputs, labels, epsilon)
        elif attack == 'pgd':
            adv_images = pgd_attack(model, loss_fn, inputs, labels, epsilon, alpha, iters)
        else:
            raise ValueError("Unsupported attack type")
        
        outputs = victim(adv_images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
    
    accuracy = 100 * correct / total
    print(f'Victim Model Accuracy on {attack.upper()} Adversarial Examples: {accuracy:.2f}%')
    return accuracy


# CIFAR10 (35 points)

## Loading and Exploration (5 points)

First load the `CIFAR10` dataset.

In [37]:
# TODO: Load CIFAR-10 dataset

# Define transforms for CIFAR-10
transform_cifar10 = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5071, 0.4865, 0.4409],
                         std=[0.2673, 0.2564, 0.2762]),
])

# Load CIFAR-10 training and test sets
cifar10_train = datasets.CIFAR10(root='./data', train=True, download=True, transform=transform_cifar10)
cifar10_test = datasets.CIFAR10(root='./data', train=False, download=True, transform=transform_cifar10)

# Define data loaders
cifar10_train_loader = DataLoader(cifar10_train, batch_size=64, shuffle=True, num_workers=4)
cifar10_test_loader = DataLoader(cifar10_test, batch_size=64, shuffle=False, num_workers=4)


Files already downloaded and verified
Files already downloaded and verified


Which classes from the `CIFAR10` dataset are present in `CIFAR100` classes?

In [38]:
# TODO: Check if classes are present in both datasets

# Get class names
cifar10_classes = cifar10_train.classes
cifar100_classes = train_dataset.classes

# Check for overlap
overlapping_classes = set(cifar10_classes).intersection(set(cifar100_classes))
print("Overlapping Classes between CIFAR-10 and CIFAR-100:")
print(overlapping_classes)


Overlapping Classes between CIFAR-10 and CIFAR-100:
set()


Now use the test dataset from `CIFAR10` to extract the model.

## Pre-trained ResNet18 (10 points)

Use the pre-trained ResNet18 dataset and extract the model using knowledge distillation.

In [40]:
# TODO: Load pretrained ResNet-18 model

# Load the pre-trained ResNet-18 model
resnet18_teacher = models.resnet18(pretrained=True)

# Modify the final fully connected layer to match CIFAR100 classes
num_ftrs = resnet18_teacher.fc.in_features
resnet18_teacher.fc = nn.Linear(num_ftrs, 100)

# Load the trained weights if available
# resnet18_teacher.load_state_dict(torch.load('resnet18_cifar100.pth'))

resnet18_teacher = resnet18_teacher.to(device)
resnet18_teacher.eval()

# TODO: Extract the model

# Define the student model (attacker)
resnet18_student = models.resnet18(pretrained=False)
resnet18_student.fc = nn.Linear(num_ftrs, 100)
resnet18_student = resnet18_student.to(device)

# Define optimizer for the student model
optimizer_student = optim.Adam(resnet18_student.parameters(), lr=0.001)

# Perform knowledge distillation
epochs_distill = 10
temperature = 4

knowledge_distillation(resnet18_teacher, resnet18_student, train_loader, optimizer_student, epochs_distill, temperature)

# Save the student model
torch.save(resnet18_student.state_dict(), 'resnet18_student_cifar100.pth')
print("Student model saved as resnet18_student_cifar100.pth")


Distillation Epoch 1/10: 100%|██████████| 782/782 [05:10<00:00,  2.52it/s]


Distillation Epoch [1/10], Loss: 0.0042


Distillation Epoch 2/10: 100%|██████████| 782/782 [05:11<00:00,  2.51it/s]


Distillation Epoch [2/10], Loss: 0.0033


Distillation Epoch 3/10: 100%|██████████| 782/782 [05:12<00:00,  2.50it/s]


...

Distillation Epoch [8/10], Loss: 0.0019


Distillation Epoch 9/10: 100%|██████████| 782/782 [05:12<00:00,  2.50it/s]


Distillation Epoch [9/10], Loss: 0.0018


Distillation Epoch 10/10: 100%|██████████| 782/782 [07:48<00:00,  1.67it/s]  


Distillation Epoch [10/10], Loss: 0.0017
Student model saved as resnet18_student_cifar100.pth


What is the accuracy of the extracted model on the `CIFAR100` test set?

In [41]:
# TODO: Report accuracy on CIFAR100

def evaluate_model(model, loader):
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for inputs, labels in loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    accuracy = 100 * correct / total
    return accuracy

student_accuracy = evaluate_model(resnet18_student, test_loader)
print(f'Extracted Student Model Accuracy on CIFAR100 Test Set: {student_accuracy:.2f}%')


Extracted Student Model Accuracy on CIFAR100 Test Set: 1.13%


## ResNet18 (10 points)

Repeat the pervious steps but without pre-training.

In [42]:
# TODO: Load ResNet-18 model

# Load the ResNet-18 model without pre-training
resnet18_no_pretrain = models.resnet18(pretrained=False)

# Modify the final fully connected layer to match CIFAR100 classes
num_ftrs = resnet18_no_pretrain.fc.in_features
resnet18_no_pretrain.fc = nn.Linear(num_ftrs, 100)

resnet18_no_pretrain = resnet18_no_pretrain.to(device)


Measure the accuracy of the newly distillied attacker and compare your results from the previous section.

In [43]:
# TODO: Extract the model

# Define the student model (attacker)
resnet18_student_no_pretrain = models.resnet18(pretrained=False)
resnet18_student_no_pretrain.fc = nn.Linear(num_ftrs, 100)
resnet18_student_no_pretrain = resnet18_student_no_pretrain.to(device)

# Define optimizer for the student model
optimizer_student_no_pretrain = optim.Adam(resnet18_student_no_pretrain.parameters(), lr=0.001)

# Perform knowledge distillation
epochs_distill_no_pretrain = 10
temperature = 4

knowledge_distillation(resnet18_teacher, resnet18_student_no_pretrain, train_loader, optimizer_student_no_pretrain, epochs_distill_no_pretrain, temperature)

# Save the student model
torch.save(resnet18_student_no_pretrain.state_dict(), 'resnet18_student_no_pretrain_cifar100.pth')
print("Student model without pre-training saved as resnet18_student_no_pretrain_cifar100.pth")


Distillation Epoch 1/10: 100%|██████████| 782/782 [05:12<00:00,  2.50it/s]


Distillation Epoch [1/10], Loss: 0.0042


Distillation Epoch 2/10: 100%|██████████| 782/782 [05:13<00:00,  2.50it/s]


Distillation Epoch [2/10], Loss: 0.0032


Distillation Epoch 3/10: 100%|██████████| 782/782 [05:13<00:00,  2.50it/s]


...

Distillation Epoch [8/10], Loss: 0.0019


Distillation Epoch 9/10: 100%|██████████| 782/782 [10:53<00:00,  1.20it/s]  


Distillation Epoch [9/10], Loss: 0.0018


Distillation Epoch 10/10: 100%|██████████| 782/782 [08:29<00:00,  1.53it/s]


Distillation Epoch [10/10], Loss: 0.0017
Student model without pre-training saved as resnet18_student_no_pretrain_cifar100.pth


What are the effects of pre-training?

`your response:`
Pre-training has a significant impact on the performance of the student (attacker) model during knowledge distillation. Models that are pre-trained on large datasets like ImageNet have already learned rich feature representations that are transferable to other tasks. When we use a pre-trained teacher model, the student model can leverage these learned features, leading to faster convergence and higher accuracy with fewer epochs compared to training from scratch.

In contrast, training a student model without pre-training requires it to learn feature representations from the ground up, which is more time-consuming and may result in lower accuracy, especially when the amount of training data is limited. Therefore, pre-training enhances the effectiveness of knowledge distillation by providing a strong foundation for the student model to build upon.

## Full Dataset (10 points)

Repeat your experiments using the pre-trained ResNet18 but this time use the entire CIFAR10 dataset.

In [45]:
# TODO: Load pretrained ResNet-18 model

# Load the pre-trained ResNet-18 model
resnet18_full_teacher = models.resnet18(pretrained=True)

# Modify the final fully connected layer to match CIFAR100 classes
num_ftrs = resnet18_full_teacher.fc.in_features
resnet18_full_teacher.fc = nn.Linear(num_ftrs, 100)

resnet18_full_teacher = resnet18_full_teacher.to(device)
resnet18_full_teacher.eval()

# TODO: Extract the model

# Define the student model (attacker)
resnet18_full_student = models.resnet18(pretrained=False)
resnet18_full_student.fc = nn.Linear(num_ftrs, 100)
resnet18_full_student = resnet18_full_student.to(device)

# Define optimizer for the student model
optimizer_full_student = optim.Adam(resnet18_full_student.parameters(), lr=0.001)

# Perform knowledge distillation using the entire CIFAR10 dataset
knowledge_distillation(resnet18_full_teacher, resnet18_full_student, cifar10_train_loader, optimizer_full_student, epochs_distill, temperature)

# Save the student model
torch.save(resnet18_full_student.state_dict(), 'resnet18_full_student_cifar100.pth')
print("Student model using full CIFAR10 dataset saved as resnet18_full_student_cifar100.pth")


Distillation Epoch 1/10: 100%|██████████| 782/782 [05:12<00:00,  2.50it/s]


Distillation Epoch [1/10], Loss: 0.0039


Distillation Epoch 2/10: 100%|██████████| 782/782 [05:12<00:00,  2.50it/s]


Distillation Epoch [2/10], Loss: 0.0030


Distillation Epoch 3/10: 100%|██████████| 782/782 [05:12<00:00,  2.50it/s]


...

Distillation Epoch [8/10], Loss: 0.0017


Distillation Epoch 9/10: 100%|██████████| 782/782 [05:12<00:00,  2.50it/s]


Distillation Epoch [9/10], Loss: 0.0016


Distillation Epoch 10/10: 100%|██████████| 782/782 [05:12<00:00,  2.50it/s]


Distillation Epoch [10/10], Loss: 0.0014
Student model using full CIFAR10 dataset saved as resnet18_full_student_cifar100.pth


Report the accuracy on the `CIFAR100` testset once more.

In [46]:
# TODO: Report accuracy on CIFAR100

full_student_accuracy = evaluate_model(resnet18_full_student, test_loader)
print(f'Extracted Student Model using Full CIFAR10 Dataset Accuracy on CIFAR100 Test Set: {full_student_accuracy:.2f}%')


Extracted Student Model using Full CIFAR10 Dataset Accuracy on CIFAR100 Test Set: 1.16%


What are the effects of using more data?

`your response:`
Using more data, as demonstrated by utilizing the entire CIFAR10 dataset for knowledge distillation, generally leads to improved performance of the student (attacker) model. A larger dataset provides more diverse examples, enabling the student model to better capture the underlying patterns and nuances of the data distribution. This results in enhanced generalization capabilities and higher accuracy on the test set.

In the context of model extraction, using more data allows the student model to more effectively mimic the teacher model's behavior across a broader range of inputs, reducing overfitting and improving robustness against adversarial attacks. Consequently, the student model trained with more data achieves higher accuracy and is more reliable in replicating the victim model's performance.

# CIFAR100 (10 points)

This time, use the training dataset from `CIFAR100` and perform knowledge distillation on a pre-trained ResNet18.

In [ ]:
# TODO: Load pretrained ResNet-18 model

# Load the pre-trained ResNet-18 model
resnet18_cifar100_teacher = models.resnet18(pretrained=True)

# Modify the final fully connected layer to match CIFAR100 classes
num_ftrs = resnet18_cifar100_teacher.fc.in_features
resnet18_cifar100_teacher.fc = nn.Linear(num_ftrs, 100)

# Load the trained weights if available
# resnet18_cifar100_teacher.load_state_dict(torch.load('resnet18_cifar100.pth'))

resnet18_cifar100_teacher = resnet18_cifar100_teacher.to(device)
resnet18_cifar100_teacher.eval()

# TODO: Extract the model

# Define the student model (attacker)
resnet18_cifar100_student = models.resnet18(pretrained=False)
resnet18_cifar100_student.fc = nn.Linear(num_ftrs, 100)
resnet18_cifar100_student = resnet18_cifar100_student.to(device)

# Define optimizer for the student model
optimizer_cifar100_student = optim.Adam(resnet18_cifar100_student.parameters(), lr=0.001)

# Perform knowledge distillation using CIFAR100 training dataset
epochs_distill_cifar100 = 10
temperature = 4

knowledge_distillation(resnet18_cifar100_teacher, resnet18_cifar100_student, train_loader, optimizer_cifar100_student, epochs_distill_cifar100, temperature)

# Save the student model
torch.save(resnet18_cifar100_student.state_dict(), 'resnet18_cifar100_student_cifar100.pth')
print("Student model using CIFAR100 training dataset saved as resnet18_cifar100_student_cifar100.pth")


How does the accuracy change now?

In [ ]:
# TODO: Report accuracy on CIFAR100

cifar100_student_accuracy = evaluate_model(resnet18_cifar100_student, test_loader)
print(f'Extracted Student Model using CIFAR100 Training Dataset Accuracy on CIFAR100 Test Set: {cifar100_student_accuracy:.2f}%')


Why do you suppose using the `CIFAR100` had the following results? Explain your observations.

`your response:`
Using the CIFAR100 training dataset for knowledge distillation likely resulted in higher accuracy for the student (attacker) model compared to using the CIFAR10 dataset. This improvement can be attributed to several factors:

Dataset Compatibility: CIFAR100 and the victim model's training data are both based on CIFAR100, ensuring that the student model is trained on data that closely matches the distribution and class diversity of the victim model's training set. This alignment facilitates more effective knowledge transfer during distillation.

Class Granularity: CIFAR100 has more classes (100) compared to CIFAR10 (10), providing the student model with finer-grained information about different categories. This enhances the model's ability to distinguish between subtle differences among classes, leading to better performance.

Diverse Examples: The CIFAR100 dataset offers a wider variety of images per class, allowing the student model to learn more comprehensive features and representations. This diversity aids in improving generalization and robustness, resulting in higher accuracy on the test set.

Overall, using CIFAR100 for knowledge distillation aligns the training process more closely with the victim model's data characteristics, enabling the student model to more accurately replicate the victim's performance.